# Proyecto de detección de fraudes de transacciones con Tarjetas de Crédito (TDC)

## Definición de problema y objetivo

About the Dataset

This is a simulated credit card transaction dataset containing legitimate and fraud transactions from the duration 1st Jan 2019 - 31st Dec 2020. It covers credit cards of 1000 customers doing transactions with a pool of 800 merchants.

### Descripción de los datos

### Métricas de evaluación

## Importando librerías

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import plotly.express as px
from datetime import datetime


## Carga de datos

In [6]:
# test_original = pd.read_csv('C:/Users/ramos/Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTest.csv')
# train_original = pd.read_csv('C:/Users/ramos/Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTrain.csv')

test_original = pd.read_csv('C:/Users/omarh\Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTest.csv')
train_original = pd.read_csv('C:/Users/omarh\Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTrain.csv')


### Conociendo el contenido de los dataframes:

In [7]:
# test_original.info()
# train_original.info()

In [8]:
#De manera intuitiva podemos saber que los dataframes tienen las mismas columnas, sin embargo realizamos una verificación
# Vamos a unificar los datos para realizar el EDA general, posteriormente vamos separar los datos de entrenamiento y prueba

raw_data = pd.concat([test_original,train_original], axis=0)
#eliminamos la primer columna "Unnamed:0" que es un duplicado del índice
raw_data = raw_data.drop(columns=raw_data.columns[0],axis=1)

raw_data.head(3)
raw_data.info(show_counts=True)


<class 'pandas.core.frame.DataFrame'>
Index: 1852394 entries, 0 to 1296674
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1852394 non-null  object 
 1   cc_num                 1852394 non-null  int64  
 2   merchant               1852394 non-null  object 
 3   category               1852394 non-null  object 
 4   amt                    1852394 non-null  float64
 5   first                  1852394 non-null  object 
 6   last                   1852394 non-null  object 
 7   gender                 1852394 non-null  object 
 8   street                 1852394 non-null  object 
 9   city                   1852394 non-null  object 
 10  state                  1852394 non-null  object 
 11  zip                    1852394 non-null  int64  
 12  lat                    1852394 non-null  float64
 13  long                   1852394 non-null  float64
 14  city_pop               

#### **Descripción del Dataframe**
tenemos un dataframe con 1,852,394 registros no nulos

##### Descripción de columnas
* trans_date_trans_time	 : Fecha de transacción 
    * tipo object, *_**será cambiado a datetime**_*
* ccnum : Número de transacción 
    * tipo int64
* merchant : nombe del comercio donde se hizo la transacción
    * tipo object
* category : categoría
    * tipo object
* amt : monto de la transacción
    * tipo float64
* first : primer nombre del titular de la tarjeta
    * tipo object
* last : apellido del titular de la tarjeta
     * tipo object
* gender: género del titular de la tarjeta
    * tipo object
* street : calle del titular
    * tipo object
* city : ciudad del titular
    * tipo object
* state :  estado del titular
    * tipo object
* zip : código postal del titular
    * tipo int64
* lat : latitud asociada a la dirección del titular
    * tipo float64 
* long : longitud asociada a la dirección del titular
    * tipo float64
* city_pop : población de la ciudad del titular de la tarjeta
    * tipo int64
* job : profesión del titular
    * tipo object
* dob : fecha de nacimiento del titular
    * tipo object, *_**será cambiado a datetime**_*
* trans_num : número único de transacción
    * tipo object
* unix_time : tiempo de la transacción en formato unix
    * tipo int64 , *_**Será eliminado**_*
* merch_lat : latitud de la ubicación del comerciante
    * tipo float64
* merch_long : longitud de la ubicación del comerciante
    * tipo float64
* is_fraud : es fraude 
    * tipo int64

## Preprocesamiento de datos

### Trabajando con dataset **raw_data**
* Eliminación de columnas innecesarias
* Conversión de tipo de datos en columnas
* Verificación de nulos
* Eliminación de duplicados
* Creación de dataframe con una muestra estratificada

In [9]:
raw_data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [10]:
#Eliminando columna 'unix_time'
raw_data = raw_data.drop(columns='unix_time')


In [11]:
#convirtiendo 'trans_date_trans_time'  a datetime
raw_data['trans_date_trans_time'] = pd.to_datetime(raw_data['trans_date_trans_time'])
display(raw_data['trans_date_trans_time'].sample(2))

raw_data['dob'] = pd.to_datetime(raw_data['dob'])
display(raw_data['dob'].sample(2))


104280   2020-07-27 16:21:06
488618   2019-08-01 21:35:00
Name: trans_date_trans_time, dtype: datetime64[ns]

411121   1977-12-16
627472   1976-09-12
Name: dob, dtype: datetime64[ns]

In [12]:
#Verificación de nulos
raw_data.isnull().any()


trans_date_trans_time    False
cc_num                   False
merchant                 False
category                 False
amt                      False
first                    False
last                     False
gender                   False
street                   False
city                     False
state                    False
zip                      False
lat                      False
long                     False
city_pop                 False
job                      False
dob                      False
trans_num                False
merch_lat                False
merch_long               False
is_fraud                 False
dtype: bool

In [13]:
#Eliminación de duplicados
print('filas antes de eliminación de duplicados:',raw_data.shape[0] )
raw2=raw_data.drop_duplicates()
print('Filas después de eliminación de duplicados',raw2.shape[0])
raw_data.shape

filas antes de eliminación de duplicados: 1852394
Filas después de eliminación de duplicados 1852394


(1852394, 21)

In [14]:
#Creando una copia del dataframe para trabajar
# data = raw_data.copy()

In [15]:
#Creando un dataframe con una muestra estratificada del 50%
sampled_data,_ = train_test_split(raw_data,
                                  test_size=0.1,
                                    stratify=raw_data['is_fraud'],
                                      random_state=1234)

In [16]:
display(sampled_data.shape)

(1667154, 21)

## Análisis exploratorio de los datos (EDA)

In [17]:
sampled_data.describe()

,trans_date_trans_time,cc_num,amt,zip,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
count,1667154,1.667154e+06,1.667154e+06,1.667154e+06,1.667154e+06,1.667154e+06,1.667154e+06,1667154,1.667154e+06,1.667154e+06,1.667154e+06
mean,2020-01-20 22:43:25.365570816,4.180821e+17,7.010478e+01,4.880415e+04,3.854036e+01,-9.022455e+01,8.870520e+04,1973-10-15 17:38:26.063147152,3.853971e+01,-9.022461e+01,5.210077e-03
min,2019-01-01 00:00:18,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1924-10-30 00:00:00,1.902742e+01,-1.666716e+02,0.000000e+00
25%,2019-07-23 06:50:04.500000,1.800429e+14,9.640000e+00,2.623700e+04,3.466890e+01,-9.679800e+01,7.410000e+02,1962-08-13 00:00:00,3.474223e+01,-9.689647e+01,0.000000e+00
50%,2020-01-02 07:08:27,3.521417e+15,4.747000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.443000e+03,1975-11-30 00:00:00,3.936982e+01,-8.743765e+01,0.000000e+00
75%,2020-07-23 12:35:07,4.642255e+15,8.311000e+01,7.201100e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1987-04-23 00:00:00,4.195641e+01,-8.024262e+01,0.000000e+00
max,2020-12-31 23:59:34,4.992346e+18,2.894890e+04,9.992100e+04,6.669330e+01,-6.795030e+01,2.906700e+06,2005-01-29 00:00:00,6.751027e+01,-6.695090e+01,1.000000e+00
std,NaN,1.310131e+18,1.606598e+02,2.687992e+04,5.071691e+00,1.374843e+01,3.016529e+05,NaN,5.105929e+00,1.376020e+01,7.199260e-02


In [18]:
sampled_data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [19]:
#Creando la columna "age" para conocer las edades de los usuarios
#creamos la fecha de hoy
today = datetime.now()
#restamos el año actual al año de nacimiento 
sampled_data['age'] = today.year - sampled_data['dob'].dt.year


In [20]:
is_fraud = sampled_data[sampled_data['is_fraud']==1]

In [21]:
#Top 10 empleos que sufrieron fraudes
top_jobs_fraud = pd.DataFrame(is_fraud[['job']].value_counts().sort_values(ascending=False).head(10))
top_jobs_fraud.reset_index(inplace=True)

# top_jobs_fraud = top_jobs_fraud.drop(columns='is_fraud')
px.bar(top_jobs_fraud, x='job',y='count',color='job', text='count',title='Top 10 empleos que sufrieron fraude')






In [22]:
#Explorando la cantidad de fraudes por ciudad
top_cities_fraud = pd.DataFrame(is_fraud[['city']].value_counts().sort_values(ascending=False).head(10))
top_cities_fraud.reset_index(inplace=True)
# top_cities_fraud.drop(columns='is_fraud',inplace=True)
px.bar(top_cities_fraud, x='city', y='count', color='city', text='count', title='Top 10 ciudades donde viven las víctimas de fraude')


In [23]:
#Explorando fraudes por categoría y género
category_fraud = pd.DataFrame(is_fraud[['category','gender']].value_counts().sort_values(ascending=False))
category_fraud.reset_index(inplace=True)
category_fraud

fig = px.bar(category_fraud, x='category',y='count',color='gender', text='count',title='Cantidad de fraudes recibidos por categoría y género')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [24]:
sum_top2=category_fraud[category_fraud['category'].isin(['shopping_net','grocery_pos'])]['count'].sum()
total_cat= category_fraud[['category','count']]['count'].sum()
# total_percent= 
print(f'Las categorias Shopping_net, grocery_pos representan el' ,round((sum_top2*100)/total_cat,2), '% del total de incidencias de fraudes')

Las categorias Shopping_net, grocery_pos representan el 46.25 % del total de incidencias de fraudes


*_**Estudiando las dos categorías más propensas a fraudes:**_*

* shopping_net
* grocery_pos

In [25]:
#Shopping_net (Compras en línea)
age_bins = [20,25,30,35,40,50,60,70,80,90,100,110]

ages_shop_net = pd.DataFrame(is_fraud[is_fraud['category']=='shopping_net']['age'].value_counts().sort_values(ascending=False))
ages_shop_net.reset_index(inplace=True)

ages_shop_net['ages_group'] = pd.cut(ages_shop_net['age'],age_bins,right=False, labels = ["20-24", "25-29", "30-34", "35-39", "40-49", "50-59",'60-69','70-79','80-89','90-99','100-109'])

ages_group_net = pd.DataFrame(ages_shop_net.groupby('ages_group',observed=True)['count'].sum())
ages_group_net.reset_index(inplace=True)
display(ages_group_net['count'].sum())
fig = px.bar(ages_group_net, x='ages_group', y ='count', color='ages_group',text='count')

fig.update_traces(texttemplate='%{text:.2s}',textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.show()

2002

In [59]:
#Shopping pos (compras en punto de venta)
# 2015
ages_grocery_pos = pd.DataFrame(is_fraud[is_fraud['category']=='grocery_pos']['age'].value_counts().sort_values(ascending=False))
ages_grocery_pos.reset_index(inplace=True)


ages_grocery_pos['ages_group'] =pd.cut(ages_grocery_pos['age'],age_bins,right=False, labels = ["20-24", "25-29", "30-34", "35-39", "40-49", "50-59",'60-69','70-79','80-89','90-99','100-109'])

ages_group_grocery_pos = pd.DataFrame(ages_grocery_pos.groupby('ages_group')['count'].sum())

ages_group_grocery_pos.reset_index(inplace=True)




px.bar(ages_group_grocery_pos, x='ages_group', y = 'count')

# ages_group_grocery_pos







C:\Users\omarh\AppData\Local\Temp\ipykernel_16636\2550982171.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [228]:
sampled_data['merchant'].head(30)

985661                       fraud_Predovic Inc
462365     fraud_Schroeder, Wolff and Hermiston
177162                       fraud_Medhurst PLC
1149772                        fraud_Stark-Batz
337575                          fraud_Brown Inc
1055318     fraud_Bahringer, Schoen and Corkery
410611                         fraud_Harris Inc
47917                         fraud_Cormier LLC
622954                          fraud_Lynch Ltd
106302                        fraud_Zboncak LLC
531988                    fraud_Deckow-O'Conner
366596                  fraud_Cartwright-Harris
534156                     fraud_O'Keefe-Wisoky
866033                      fraud_Jast and Sons
201801           fraud_Witting, Beer and Ernser
1201990                      fraud_Homenick LLC
1027850                       fraud_Gerlach Inc
813680       fraud_Windler, Goodwin and Kovacek
875439       fraud_Windler, Goodwin and Kovacek
1068440                        fraud_Jacobi Inc
161343                       fraud_Hickl

In [229]:
is_fraud['merchant'].value_counts()

merchant
fraud_Kilback LLC                      60
fraud_Kozey-Boehm                      59
fraud_Rau and Sons                     54
fraud_Doyle Ltd                        53
fraud_Kiehn-Emmerich                   51
                                       ..
fraud_Kilback, Nitzsche and Leffler     1
fraud_Fadel Inc                         1
fraud_Windler LLC                       1
fraud_Turner LLC                        1
fraud_Boehm, Block and Jakubowski       1
Name: count, Length: 683, dtype: int64

* La fecha mínima que tenemos de datos es el 01/01/2019
* La fecha máxima que tenemos es 31/12/2020
* los montos de transacción van desde los 70 dls hasta los 28,948
* Tenemos usuarios nacidos desde 1924 hasta 2005 (desde los 20 hasta los 101 años de edad al día de hoy)

Podemos observar que los empleos más propensos a fraudes fueron:
* Quantity Surveyor - 64 incidencias
* Materials Engineer - 58 incidencias
* Audiological Scientist - 55 incidencias
* Naval architec - 53 incidencias
* Trading standards officer - 51 incidencias

Las categorías con mayor cantidad de fraudes fueron:

* Shooping network - 2,005 incidencias
* Grocery point of sale - 2,015 incidencias
* Miscellaneous network - 1060 incidencias
* Shopping point of sale - 940 incidencias

* Las categorias Shopping_net, grocery_pos representan el 46.25 % del total de fraudes.
* Podemos observar que las compras en línea sufrieron 50% más de fraudes, esto se puede deber a la falta de alfabetización de los usuarios en el uso de plataformas de venta online, donde los usuarios no tengan cuidado  o conocimiento de peligros como el scam, pishing, robo de datos personales, etc.


Podemos observar que los rangos de edades con más incidencias a fraudes en las ventas online "shopping_net" fueron :
* 50-59 años - **360 incidencias**
* 60-69 años - **350 incidencias**
* 40-49 años - **300 incidencias**

Esto soporta la teoría de que las personas mayores pueden estar teniendo un problema de alfabetización en los servicios digitales y son un perfil vulnerable para los fraudes online.








### Zona de pruebas

In [230]:
# sampled_data[sampled_data['merch_long']=='-84.938483']

In [231]:
# filtered_data = sampled_data[(sampled_data['merch_lat'] == 30.630093) & (sampled_data['merch_long'] == -84.938483)]
# filtered_data


In [232]:
# #explorando ubicación del fraude
# fraud_location= pd.DataFrame(sampled_data[sampled_data['is_fraud']==1][['merch_long','merch_lat','is_fraud']].value_counts().sort_values(ascending=False))
# # .head(10))
# fraud_location.reset_index(inplace=True)
# fraud_location.drop(columns='is_fraud',inplace=True)
# fraud_location.sample(10)


In [233]:
# # Explorando la frecuencia de fraudes por ciudad
# fig = px.bar(sampled_data['job'])
# fig.show()

In [234]:
# sampled_data2 = sampled_data.copy()

In [235]:
# sampled_data2['amt'] = sampled_data2['amt'] +1
# sampled_data2['log_amt'] = np.log(sampled_data2['amt'])
# fig = px.histogram(sampled_data2['log_amt'])
# fig.show()

In [236]:
# sampled_data2['log_amt'].hist(bins=50)
# plt.title('Distribución Logarítmica de Montos de Transacciones')
# plt.xlabel('Logaritmo del Monto')
# plt.ylabel('Frecuencia')
# plt.show()

In [237]:
# #explorando la distribución de los montos de transacción
# fig = px.histogram(sampled_data['amt'])
# fig.show()

### Unificando y analizando dataframes

### Conclusiones EDA

## Creando dataset para el modelo

### Imputación de nulos

## Análisis y Selección de características (Feature engineering)

## Codificación y estandarización de los datos

### Categorización de datos 
(Onehot por ejemplo)
evaluar mantener una copia de datos sin escalar

### Escalando datos

## Manejo de desabalanceo de datos

## Creación del modelo y modelo benchmark